In [9]:
from colormath.color_objects import LabColor, XYZColor
from colormath.color_conversions import convert_color
from colormath.color_diff import delta_e_cie2000
from scipy.integrate import simps
import pandas as pd
import numpy as np

wavelength_range = [360,830]
def load_cmf(filename):
    data = pd.read_csv(filename, header=None, names=['wavelength', 'x', 'y', 'z'])
    cmf_wavelengths = data['wavelength'].values
    cmf_indices = np.where((cmf_wavelengths >= wavelength_range[0]) & (cmf_wavelengths <= wavelength_range[1]))[0]
    cmf_wavelengths = cmf_wavelengths[cmf_indices]
    x = np.interp(cmf_wavelengths, data['wavelength'].values, data['x'].values)
    y = np.interp(cmf_wavelengths, data['wavelength'].values, data['y'].values)
    z = np.interp(cmf_wavelengths, data['wavelength'].values, data['z'].values)
    cmf = {
        'wavelength': cmf_wavelengths,
        'x': x,
        'y': y,
        'z': z,
    }
    return cmf

def load_spd(filename, cmf):
    data = pd.read_csv(filename, sep='\t', skiprows=2, header=None, names=['wavelength', 'intensity'])
    spd_wavelengths = np.round(data['wavelength']).astype(int).values
    spd_indices = np.where((spd_wavelengths >= wavelength_range[0]) & (spd_wavelengths <= wavelength_range[1]))[0]
    spd_wavelengths = spd_wavelengths[spd_indices]
    spd_intensities = data['intensity'].values[spd_indices] 
    cmf_wavelengths = cmf['wavelength']
    x = np.interp(cmf_wavelengths, spd_wavelengths, spd_intensities)

    spd = {
        'wavelength': cmf_wavelengths,
        'intensity': x,
    }
    return spd

def spd_to_xyz(spd, wavelength, cmf):
    X = simps(spd * cmf['x'], wavelength)
    Y = simps(spd * cmf['y'], wavelength)
    Z = simps(spd * cmf['z'], wavelength)
    return np.array([X, Y, Z])


def xyz_to_lab(xyz):
    xyz_color = XYZColor(*xyz)
    lab_color = convert_color(xyz_color, LabColor)
    return lab_color

def color_difference(filename1, filename2, cmf_data):
    spd_data1 = load_spd(filename1, cmf_data)
    spd_data2 = load_spd(filename2, cmf_data)

    xyz1 = spd_to_xyz(spd_data1['intensity'], spd_data1['wavelength'], cmf_data)
    xyz2 = spd_to_xyz(spd_data2['intensity'], spd_data2['wavelength'], cmf_data)

    lab1 = xyz_to_lab(xyz1)
    lab2 = xyz_to_lab(xyz2)

    color_diff = delta_e_cie2000(lab1, lab2)
    return color_diff



iphone_model = []
color_diff_set = []
# 示例：计算iPhone 11和iPhone 14之间的色差
cmf_data = load_cmf('CIE_cc_1931_2deg.csv')

for i in range (11,15):
    if i <= 13:
        dataset_filename_1 = f"{i}_white.txt"
        dataset_filename_2 = f"{i+1}_white.txt"
        color_diff = color_difference(dataset_filename_1, dataset_filename_2, cmf_data)
        iphone_model.append(f"iPhone {i} - iPhone {i+1}")
        color_diff_set.append(color_diff)
        if i<= 12:
            dataset_filename_1 = f"{i}_white.txt"
            dataset_filename_2 = f"{i+2}_white.txt"
            color_diff = color_difference(dataset_filename_1, dataset_filename_2, cmf_data) 
            iphone_model.append(f"iPhone {i} - iPhone {i+2}")
            color_diff_set.append(color_diff)          
            if i<= 11:
                dataset_filename_1 = f"{i}_white.txt"
                dataset_filename_2 = f"{i+3}_white.txt"
                color_diff = color_difference(dataset_filename_1, dataset_filename_2, cmf_data) 
                iphone_model.append(f"iPhone {i} - iPhone {i+3}")
                color_diff_set.append(color_diff)   

table = pd.DataFrame(data={"Phone Model": iphone_model, "Color Difference (%)": color_diff_set})
print(table)


             Phone Model  Color Difference (%)
0  iPhone 11 - iPhone 12             11.103170
1  iPhone 11 - iPhone 13             29.131734
2  iPhone 11 - iPhone 14              6.546534
3  iPhone 12 - iPhone 13             18.349187
4  iPhone 12 - iPhone 14              4.974987
5  iPhone 13 - iPhone 14             23.159171
